<a href="https://colab.research.google.com/github/Pulsar-kkaturi/DL-Education/blob/master/VisionDL_Lecture/Lecture3_Pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Pre-Processing

## 1. Environment Setting



In [ ]:
# 데이터셋을 이 세션으로 불러오기
!git clone https://github.com/Pulsar-kkaturi/DL-Education.git

### 1.1. Library Setting
* 파이썬 라이브러리를 불러오자

In [ ]:
# 의료영상 처리를 위한 라이브러리 설치
!pip install simpleitk

In [ ]:
import numpy as np
import os
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
import SimpleITK as sitk
import json
import csv
import pandas as pd
from sklearn import metrics as skmet
from skimage import morphology
from skimage import measure
from skimage import exposure
from skimage.transform import rotate
from skimage import io as sio
from skimage import color as skc
import cv2 as cv
from keras.preprocessing import image as kimg

### 1.2. Data Loading
* 전처리 실습에 사용할 'test.dcm' 파일을 불러오자.
* 이때 'test.dcm' 파일을 업로드해두지 않았다면 실행 시 에러가 발생할 것이므로 반드시 'test.dcm'을 올리고 실행 시켜야 한다.

In [ ]:
test_data = './DL-Education/dataset/test.dcm'
image = sitk.ReadImage(test_data)
img_arr = sitk.GetArrayFromImage(image)
print('# Header Information #')
print('Image Size = ', image.GetSize())
print('Pixel Spacing = ', image.GetSpacing())
print('Image Dimension = ', image.GetDimension())
print('Number of Pixel Components = ', image.GetNumberOfComponentsPerPixel())
print('Minimum & Maximum pixel value(Min/Max) = {}/{}'.format(np.min(img_arr), np.max(img_arr)))
print('Image mean & std = {}, {}'.format(np.mean(img_arr), np.std(img_arr)))

plt.figure(figsize=(10,10))
plt.imshow(img_arr[0], cmap='gray')

## 2. Basic Pre-Processing
* 가장 기본적인 전처리들을 알아보자

### 2.1 Resize
* 이미지의 크기를 변화 시키는 모듈

In [ ]:
# Resize
def resize_array(sitk_image, size,interpolator=sitk.sitkLinear):
    original_spacing = sitk_image.GetSpacing()
    original_size = sitk_image.GetSize()
    new_size = list(original_size)
    new_size[0]=size[0]
    new_size[1]=size[1]
    new_spacing = [(ospc * osz / nsz) for osz, ospc, nsz in
                   zip(original_size, original_spacing, new_size)]
    sitk_image = sitk.Resample(sitk_image, new_size, sitk.Transform(), interpolator, sitk_image.GetOrigin(), new_spacing,
                         sitk_image.GetDirection(), 0, sitk_image.GetPixelID())
    return sitk_image

resize_img = resize_array(image, [128, 128])
print('Original Image Size = ', image.GetSize())
print('Processed Image Size = ', resize_img.GetSize())
print('Original Pixel Spacing = ', image.GetSpacing())
print('Processed Pixel Spacing = ', resize_img.GetSpacing())

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(img_arr[0], cmap='gray')
plt.subplot(1,2,2)
plt.imshow(sitk.GetArrayFromImage(resize_img)[0], cmap='gray')

### 2.2 Resample
* 이미지의 픽셀 간 물리적 거리를 변화시키는 모듈

In [ ]:
# Resample
def resample_array(sitk_image, spacing, interpolator=sitk.sitkLinear):
    original_spacing = sitk_image.GetSpacing()
    original_size = sitk_image.GetSize()
    new_spacing = [spacing, spacing, original_spacing[2]]
    new_size = [int(round(osz * ospc / nspc)) for osz, ospc, nspc in
                zip(original_size, original_spacing, new_spacing)]
    sitk_image = sitk.Resample(sitk_image, new_size, sitk.Transform(), interpolator, sitk_image.GetOrigin(), new_spacing,
                         sitk_image.GetDirection(), 0, sitk_image.GetPixelID())
    return sitk_image

resample_img = resample_array(image, 0.5)
print('Original Pixel Spacing = ', image.GetSpacing())
print('Processed Pixel Spacing = ', resample_img.GetSpacing())
print('Original Image Size = ', image.GetSize())
print('Processed Image Size = ', resample_img.GetSize())

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(img_arr[0], cmap='gray')
plt.subplot(1,2,2)
plt.imshow(sitk.GetArrayFromImage(resample_img)[0], cmap='gray')

### 2.3 Min-Max Normalization
* 이미지의 픽셀 값들을 0과 1사이의 값으로 바꾸어 정규화시키는 모듈

In [ ]:
# MinMax Normalization
norm_img = (img_arr - np.min(img_arr)) / (np.max(img_arr) - np.min(img_arr))
print('Oringinal Image min/max value = {}/{}'.format(np.min(img_arr), np.max(img_arr)))
print('Processed Image min/max value = {}/{}'.format(np.min(norm_img), np.max(norm_img)))
print('\nSample Patch Comparison(Origin vs Processed)')
print(img_arr[0, 185:190, 275:280])
print(norm_img[0, 185:190, 275:280])

plt.figure(figsize=(8,8))
plt.imshow(norm_img[0], cmap='gray')

### 2.4 Z-Score Normalization
* Z-score를 이용하여 이미지의 픽셀 값들을 정규화시키는 모듈

In [ ]:
# Z-Score Normalization
zsc_img = (img_arr - np.mean(img_arr)) / np.std(img_arr)
print('Oringinal Image min/max value = {}/{}'.format(np.min(img_arr), np.max(img_arr)))
print('Processed Image min/max value = {}/{}'.format(np.min(zsc_img), np.max(zsc_img)))
print('Oringinal Image mean/std value = {}/{}'.format(np.mean(img_arr), np.std(img_arr)))
print('Processed Image mean/std value = {}/{}'.format(np.mean(zsc_img), np.std(zsc_img)))
print('\nSample Patch Comparison(Origin vs Processed)')
print(img_arr[0, 185:190, 275:280])
print(zsc_img[0, 185:190, 275:280])

plt.figure(figsize=(8,8))
plt.imshow(zsc_img[0], cmap='gray')

### 2.5 Gamma Correction
* 이미지의 감마 값을 조정하는 모듈

In [ ]:
# Gamma Correction
gamma_cor = exposure.adjust_gamma(255*norm_img, 0.1)
print('Oringinal Image mean/min/max value = {}/{}'.format(np.mean(255*norm_img), np.std(255*norm_img)))
print('Processed Image mean/min/max value = {}/{}'.format(np.mean(gamma_cor), np.std(gamma_cor)))

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(img_arr[0], cmap='gray')
plt.subplot(1,2,2)
plt.imshow(gamma_cor[0], cmap='gray')

### 2.6 Adaptive Equalization
* 히스토그램 평활화를 수행하는 모듈

In [ ]:
# Adaptive Equalization
img_norm = (img_arr - np.min(img_arr))/(np.max(img_arr)-np.min(img_arr))
img_eqh = exposure.equalize_adapthist(img_norm, clip_limit=0.05)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(img_arr[0], cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_eqh[0], cmap='gray')

## 3. Hounsfield Unit Lung Area Segmentation
* 다양한 영상처리 기법들을 조합하여 폐 CT 영상에서 뼈와 다른 장기들을 제외하고 오직 페영역만 분할해보자.

### 3.1 Hu Conversion
* Hounsfield 값으로 이미지의 픽셀 값들을 변환하자.

In [ ]:
itc = image.GetMetaData('0028|1052')
slp = image.GetMetaData('0028|1053')
print('Rescale Slope (0028|1053) = ', image.GetMetaData('0028|1053'))
print('Rescale Intercept (0028|1052) = ', image.GetMetaData('0028|1052'))

In [ ]:
img_arr_hu = (img_arr * float(slp)) + float(itc)
print(img_arr.shape, img_arr_hu.shape)
print('original = ', np.min(img_arr), np.max(img_arr))
print('HU conversion = ', np.min(img_arr_hu), np.max(img_arr_hu))
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.imshow(img_arr[0], cmap='gray')
plt.title('Original', fontsize=15)
plt.subplot(1,2,2)
plt.imshow(img_arr_hu[0], cmap='gray')
plt.title('HU Conversion', fontsize=15)

### 3.2 Window Setting
* 변화된 Hounsfield 값들을 기준으로 뼈와 같이 지나치게 밝은 부분과 공기와 지방과 같이 지나치게 어두운 부분들은 제외시키는 모듈

In [ ]:
img_arr_win1 = np.where(img_arr_hu < -2000, -2000, img_arr_hu)
img_arr_win = np.where(img_arr_win1 > 150, 150, img_arr_win1)

plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(img_arr[0], cmap='gray')
plt.title('Original', fontsize=15)
plt.subplot(132)
plt.imshow(img_arr_hu[0], cmap='gray')
plt.title('HU Conversion', fontsize=15)
plt.subplot(133)
plt.imshow(img_arr_win[0], cmap='gray')
plt.title('Window Setting', fontsize=15)

### 3.3 Make Mask
* Thresholding된 이미지를 기준으로 폐영역만 나타내는 마스크를 만드는 모듈

In [ ]:
# Thresholding
img_arr_thres = np.where(img_arr_win < -1200, 1, 0)
print(np.min(img_arr_thres),np.max(img_arr_thres))

plt.figure(figsize=(8,8))
plt.imshow(img_arr_thres[0], cmap='gray')
plt.title('Threshold', fontsize=15)

In [ ]:
# Erosion
eroded = morphology.erosion(img_arr_thres[0], morphology.disk(3))

plt.figure(figsize=(8,8))
plt.imshow(eroded, cmap='gray')
plt.title('Eroded', fontsize=15)

In [ ]:
# Dilation
dilation = morphology.dilation(eroded,morphology.disk(10))

plt.figure(figsize=(8,8))
plt.imshow(dilation, cmap='gray')
plt.title('Dilation', fontsize=15)

In [ ]:
# Closing
closing = morphology.closing(dilation, morphology.disk(10))

plt.figure(figsize=(8,8))
plt.imshow(closing, cmap='gray')
plt.title('closing', fontsize=15)

In [ ]:
# color label
labels = measure.label(closing)
show_lab = np.where(labels==0, 1, 0)

plt.figure(figsize=(8,8))
plt.imshow(labels, cmap='viridis')
plt.title('Color Labels', fontsize=15)

In [ ]:
# find good label
print(np.max(labels))
row_size= labels.shape[0]
col_size = labels.shape[1]
regions = measure.regionprops(labels)
good_labels = []
for prop in regions:
    B = prop.bbox
    if B[2]-B[0]<row_size*0.9 and B[3]-B[1]<col_size*0.9 and B[0]>row_size*0.1 and B[2]>col_size*0.1:
        good_labels.append(prop.label)
mask = np.zeros(shape=(row_size,col_size),dtype=np.int8)
for N in good_labels:
    mask = mask + np.where(labels==N,1,0)
mask = morphology.dilation(mask, morphology.disk(5)) #mask = np.where(labels == 0, 1, 0)
mask = morphology.closing(mask, morphology.disk(5))
mask = morphology.erosion(mask, morphology.disk(10))

print(np.min(mask), np.max(mask))
plt.figure(figsize=(8,8))
plt.imshow(mask, cmap='gray')
plt.title('Final Mask', fontsize=15)

### 3.4 Lung Area Segmentation
* 만들어진 폐영역 분할 마스크를 가지고 페 CT 영상에서 폐영역만 정확하게 분할하는 모듈

In [ ]:
# final image
img_arr_norm = img_arr_win+2000
lung_img = img_arr_norm[0] * mask

plt.figure(figsize=(8,8))
plt.imshow(lung_img, cmap='gray')
plt.title('Final Lung Image', fontsize=15)

In [ ]:
# Adaptive Equalization
rescale_image = (lung_img - np.mean(lung_img)) / (np.max(lung_img) - np.mean(lung_img))
aeh_img = exposure.equalize_adapthist(rescale_image, clip_limit=0.02)

plt.figure(figsize=(8,8))
plt.imshow(aeh_img, cmap='gray', vmin=-1, vmax=1)
plt.title('CLAHE Lung Image', fontsize=15)

## 4. Data Augmentation
* 다양한 데이터 증대(Augmentation) 기법들을 살펴보자.

In [ ]:
img_rot = rotate(img_arr[0], 10)
plt.figure(figsize=(8,8))
plt.imshow(img_rot, cmap='gray')
plt.show()

In [ ]:
img_shift = kimg.random_shift(np.array(img_arr), 0.1, 0.1)
plt.figure(figsize=(8,8))
plt.imshow(img_shift[0], cmap='gray')
plt.show()

In [ ]:
img_hflip = np.fliplr(img_arr[0])
plt.figure(figsize=(8,8))
plt.imshow(img_hflip, cmap='gray')
plt.show()

In [ ]:
img_vflip = np.flipud(img_arr[0])
plt.figure(figsize=(8,8))
plt.imshow(img_vflip, cmap='gray')
plt.show()

In [ ]:
img_shift1 = kimg.random_shift(img_arr, 0.1, 0.1)
img_rot1 = rotate(img_shift1[0], 10)
img_hflip1 = np.fliplr(img_rot1)
img_vflip1 = np.flipud(img_hflip1)
plt.figure(figsize=(8,8))
plt.imshow(img_vflip1, cmap='gray')
plt.show()

## 5. Data Simulation

* 2차원 행렬 만들기

In [ ]:
arr = [[0,0,1,0,0], [0,1,2,1,0], [1,2,3,2,1], [0,1,2,1,0], [0,0,1,0,0]]
print(arr)

* numpy array로 변환

In [ ]:
narr = np.array(arr)
print(narr.shape)
print(narr)

In [ ]:
plt.imshow(narr, cmap='gray')

* scikit-image로 이미지 크기 resize

(5 x 5) -> (128 x 128)

In [ ]:
import skimage.transform as skit
import skimage.io as skio

In [ ]:
rarr = skit.resize(narr, (128, 128))
plt.imshow(rarr, cmap='gray')

In [ ]:
# 크기에 따른 resize 변화 확인
plt.figure(figsize=(12,8))
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.title(f'size = {8*2**i} x {8*2**i}')
  plt.imshow(skit.resize(narr, (8*2**i, 8*2**i)), cmap='gray')

* 0~255 사이로 pixel value 변환

In [ ]:
print(np.min(rarr), np.max(rarr))
marr = (rarr - np.min(rarr))/(np.max(rarr)-np.min(rarr)) * 255
marr = marr.astype(np.uint8)
print(np.min(marr), np.max(marr))
plt.imshow(marr)

* 이미지 파일 png 포맷으로 저장하기

In [ ]:
carr = skc.gray2rgba(marr) # png로 저장하기 위해 RGB형식으로 변환
sio.imsave('cross.png', carr)